In [4]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [59]:
# 뉴스 날짜, 언론사, 제목 크롤링
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
query = query.replace(' ', '+')

ds = input('시작 날짜를 입력해주세요(ex.2021.01.01) : ') 
de = input('끝 날짜를 입력해주세요(ex.2021.01.31) : ')

# 정렬(sort) : 오래된순 / 유형(field) : 제목
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&sm=tab_srt&sort=0&photo=0&field=1&reporter_article=&pd=3&ds={}&de={}&docid=&nso=&mynews=1'

req = requests.get(news_url.format(query, ds, de))
soup = BeautifulSoup(req.text, 'html.parser')

news_dict = {}
idx = 0
cur_page = 1

print()
print('---크롤링 시작---')

while idx < news_num:
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    span_list = [area.find('span', {'class' : 'info'}) for area in area_list]
    press = [area.find('a', {'class' : 'info press'}) for area in area_list]
    i = 0
    i_n = len(span_list)
    word = '언론사 선정' # 언론사 이름 뒤에 같이 나오는 경우가 있어서 삭제 해줌
            
    for n in a_list[:min(len(a_list), news_num-idx)]:
        p = press[i].get_text()
        if i < i_n-1:
            if word in p:
                p = p[:-6]
            
            news_dict[idx] = {'date' : span_list[i].get_text(), 'press' : p , 'title' : n.get('title')} 
            print(idx, '번째 뉴스를 추출을 완료했습니다')
            
        idx += 1
        i += 1
    cur_page += 1

    pages = soup.find('div', {'class' : 'sc_page_inner'})
    
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')

print('---크롤링 종료---')

news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
csv_name = '네이버뉴스_{}_{}.csv'.format(query, date)

news_df.to_csv(csv_name, encoding = 'utf-8-sig')

# print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
# os.startfile(folder_path)

검색 키워드를 입력하세요 : 코로나 확진자 발생
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 20
시작 날짜를 입력해주세요(ex.2021.01.01) : 2021.03.25
끝 날짜를 입력해주세요(ex.2021.01.31) : 2021.03.26

---크롤링 시작---
0 번째 뉴스를 추출을 완료했습니다
1 번째 뉴스를 추출을 완료했습니다
2 번째 뉴스를 추출을 완료했습니다
3 번째 뉴스를 추출을 완료했습니다
4 번째 뉴스를 추출을 완료했습니다
5 번째 뉴스를 추출을 완료했습니다
6 번째 뉴스를 추출을 완료했습니다
7 번째 뉴스를 추출을 완료했습니다
8 번째 뉴스를 추출을 완료했습니다
10 번째 뉴스를 추출을 완료했습니다
11 번째 뉴스를 추출을 완료했습니다
12 번째 뉴스를 추출을 완료했습니다
13 번째 뉴스를 추출을 완료했습니다
14 번째 뉴스를 추출을 완료했습니다
15 번째 뉴스를 추출을 완료했습니다
16 번째 뉴스를 추출을 완료했습니다
17 번째 뉴스를 추출을 완료했습니다
18 번째 뉴스를 추출을 완료했습니다
---크롤링 종료---


In [60]:
import pandas as pd

croawing = pd.read_csv(xlsx_file_name)
croawing = croawing.drop(columns = ['Unnamed: 0'])
croawing

,date,press,title
0,2021.01.02.,프레시안,"해남군, 코로나 19 확진자 3명 추가 발생"
1,2021.01.02.,파이낸셜뉴스,순천서 이틀새 코로나19 확진자 11명 발생...누적 202명
2,2021.01.02.,연합뉴스,서울 오후 6시까지 코로나19 신규 확진자 185명 발생
3,2021.01.02.,노컷뉴스,손흥민의 토트넘에서도 코로나19 확진자 발생
4,2021.01.02.,매일신문,경북 안동 코로나19 확진자 1명 발생
5,2021.01.02.,쿠키뉴스,서울 코로나19 신규 발생 185명…시민 500명 중 1명이 확진자
6,2021.01.02.,연합뉴스,원주서 코로나19 확진자 15명 발생…강원도내 33명(종합)
7,2021.01.02.,연합뉴스,서울 오후 9시까지 코로나19 신규 확진자 192명 발생
8,2021.01.02.,부산일보,경남에서 코로나19 확진자 14명 발생…도내 누적 1371명
9,2021.01.02.,연합뉴스,서울 오후 6시까지 코로나19 신규 확진자 185명 발생
